In [10]:
import datetime
import numpy as np
from tqdm.notebook import tqdm
from huge_dataset_project.InputLoader import InputFile
from huge_dataset_project.PickLoader import PickFile

In [11]:
year = 2012
associations_file = f"../../data/detections/{year}/matchups_clusters_loc_adjusted_experiment.csv"

# ground truth of evaluation
pick_file = "/home/plerolland/Documents/Seismic_catalogs/Indian/2012/2012_SEIR_12Dec_29S.pick"
input_file = "/home/plerolland/Documents/Seismic_catalogs/Indian/2012/2012_SEIR_Network.input"

max_d_time, max_d_space = 20, 0.3

In [12]:
with open(associations_file, "r") as f:
    lines = [[l.strip() for l in line.split(",")] for line in f.readlines()]
for i, line in enumerate(lines):
    line[0] = datetime.datetime.strptime(line[0], "%Y%m%d_%H%M%S")
    line[1], line[2] = float(line[1]), float(line[2])
    line[9] = list(line[9:])
    lines[i] = line[:10]
lines = np.array(lines, dtype=np.object_)

picks = PickFile(pick_file)
inputs = InputFile(input_file)
inputs.hydrophone_letters = list(inputs.hydrophone_letters)

In [13]:
date_picks, loc_picks = [], []
for pick in tqdm(picks.picks):
    date, loc = pick.est_source_time, pick.est_source_pos
    date_picks.append(date)
    loc_picks.append(loc)
    
loc_picks, date_picks = np.array(loc_picks), np.array(date_picks)
date_d = np.array([[d.total_seconds() for d in d_] for d_ in lines[:,np.newaxis, 0] - date_picks])
loc_d = np.sqrt(np.sum((lines[:,np.newaxis, 1:3] - loc_picks)**2, axis=2).astype(np.float32))

valid_dates = np.abs(date_d) < max_d_time
valid_locs = loc_d < max_d_space

inside_dates = np.logical_and(lines[:, 0] > np.min(date_picks) - datetime.timedelta(seconds=max_d_time),
                              lines[:, 0] < np.max(date_picks) + datetime.timedelta(seconds=max_d_time))

  0%|          | 0/76 [00:00<?, ?it/s]

In [14]:
TP = np.any(np.logical_and(valid_dates, valid_locs), axis=1)
TPl = lines[TP]

P = np.logical_and(inside_dates, np.any(valid_locs, axis=1))
Pl = lines[P]

FP = np.logical_and(P, ~TP)
FPl = lines[FP]

FN = np.all(~np.logical_and(valid_dates, valid_locs), axis=0)
FNl = date_picks[FN]

print(f'o\\r P N\nP  {len(TPl)} {len(FPl)}\nN  {len(FNl)} {0}')

o\r P N
P  13 8
N  63 0


In [121]:
for line in FPl:
    if line[0] in TPl[:,0]:
        continue
    print(line[:3],"\n",line[9])
    print("---")

[datetime.datetime(2012, 12, 12, 1, 17, 4) -29.0083 74.6757] 
 ['NEAMS-2012', '20121212_012653', 'SWAMS-2012', '20121212_013441', 'WKER1-2012', '20121212_014324', 'H08S1-2012', '20121212_014355']
---
[datetime.datetime(2012, 12, 12, 1, 34, 38) -29.1247 74.7258] 
 ['NEAMS-2012', '20121212_014423', 'WKER1-2012', '20121212_020052', 'NCRO3-2012', '20121212_020130', 'SWAMS-2012', '20121212_015204']
---
[datetime.datetime(2012, 12, 12, 1, 46, 19) -29.2395 74.5279] 
 ['MAD-2012', '20121212_020500', 'NCRO3-2012', '20121212_021258', 'WKER1-2012', '20121212_021219', 'SWAMS-2012', '20121212_020336']
---
[datetime.datetime(2012, 12, 12, 3, 47, 5) -28.9444 74.5689] 
 ['WKER1-2012', '20121212_041330', 'NCRO3-2012', '20121212_041357', 'H08N1-2012', '20121212_041546', 'RAMA-2012', '20121212_041917', 'H08S1-2012', '20121212_041344']
---
[datetime.datetime(2012, 12, 12, 7, 31, 4) -28.868 74.4768] 
 ['NEAMS-2012', '20121212_074110', 'WKER1-2012', '20121212_075736', 'NCRO3-2012', '20121212_075750', 'H08N1

In [148]:
for pick in tqdm(picks.picks):
    date = pick.est_source_time
    stations = [inputs.hydrophone_names[inputs.hydrophone_letters.index(h)] for h in pick.hydrophones if h in inputs.hydrophone_letters]
    if date in FNl:
        for i in range(len(stations)):
            print(f"{stations[i]} - {pick.detection_times[i]}")
    print("----")

  0%|          | 0/76 [00:00<?, ?it/s]

NEAMS - 2012-12-12 00:55:43.779000
SWAMS - 2012-12-12 01:03:09.889000
WKER2 - 2012-12-12 01:11:38.974000
WKER1 - 2012-12-12 01:11:54.095000
WKER3 - 2012-12-12 01:11:58.519000
NCRO3 - 2012-12-12 01:12:24.907000
RAMA - 2012-12-12 01:18:10.379000
----
NEAMS - 2012-12-12 01:00:21.112000
SWAMS - 2012-12-12 01:07:47.222000
WKER2 - 2012-12-12 01:16:16.156000
WKER1 - 2012-12-12 01:16:31.928000
WKER3 - 2012-12-12 01:16:34.852000
NCRO3 - 2012-12-12 01:17:00.753000
----
----
NEAMS - 2012-12-12 01:12:20.546000
SWAMS - 2012-12-12 01:20:03.371000
MAD - 2012-12-12 01:21:16.834000
WKER2 - 2012-12-12 01:28:37.565000
WKER1 - 2012-12-12 01:28:52.195000
WKER3 - 2012-12-12 01:28:56.252000
NCRO3 - 2012-12-12 01:29:24.144000
RAMA - 2012-12-12 01:34:39.746000
----
NEAMS - 2012-12-12 01:14:15.746000
SWAMS - 2012-12-12 01:21:53.809000
MAD - 2012-12-12 01:23:09.255000
WKER2 - 2012-12-12 01:30:26.364000
WKER1 - 2012-12-12 01:30:42.628000
WKER3 - 2012-12-12 01:30:46.552000
NCRO3 - 2012-12-12 01:31:13.936000
RAMA -

In [122]:
detections_file = f"../../data/detections/{year}/detections.npy"  # obtained from associator_preprocess
detections = np.load(detections_file, allow_pickle=True).item()

In [152]:
stations = list(detections.keys())
name_to_s = {s.name + "-" + str(s.date_start.year) : s for s in stations}
d, s = datetime.datetime(2012,12,12,1,16,23), "WKER2-2012"
dets = detections[name_to_s[s]]
closest = np.argmin(np.abs(dets[:,0] - d))
print(dets[closest])

[datetime.datetime(2012, 12, 12, 1, 16, 22, 435897) 0.05698]
